In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Mengimpor library yang akan digunakan dalam melakukan pre-processing data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#Library untuk model
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.layers.recurrent import GRU
from keras.layers import Flatten, Conv1D, MaxPool1D
from keras.callbacks import ModelCheckpoint

In [ ]:
df = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df.head()

In [ ]:
#Melihat detail dari setiap kolom
df.info()

In [ ]:
#Membuat data dengan label untuk melakukan visualisasi
df_label = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df_label = df_label.drop('id', axis=1)
df_label['bmi'] = df['bmi']

In [ ]:
# Melakukan visualisasi dari catplot untuk kolom smoking_status dan kolom stroke

sns.catplot(x="smoking_status", y="stroke", kind="bar", data=df)
plt.figure(figsize=(25, 25))
plt.show()

In [ ]:
# Visualisasi data pasien berdasarkan jenis kelamin/gender

sns.countplot(x="gender", data=df)
plt.figure(figsize=(5, 5))
plt.show()

In [ ]:
# Menampilkan histogram dari bmi
sns.histplot(x='bmi', data=df_label)

In [ ]:
#Menampilkan bmi(body mass index) berdasarkan gender dalam plot area yang sama
sns.histplot(x='bmi', data=df_label[df_label["gender"]=="Male"], linewidth=0.01, color="royalblue")
sns.histplot(x='bmi', data=df_label[df_label["gender"]=="Female"], linewidth=0.01, color="red")

In [ ]:
# Membuat countplot untuk kolom work_type
sns.countplot(x="work_type", data=df_label, color="royalblue")

In [ ]:
# Membuat countplot work_type berdasarkan gender
sns.countplot(x="work_type", data=df_label, hue="gender")# set hue=gender

In [ ]:
#Menghilangkan kolom id karena tidak digunakan
df = df.drop('id', axis=1)

In [ ]:
#Menampilkan deskriptif statistik dari dataset sebelum di lakukan preprocessing
df.describe()

In [ ]:
#Mendeteksi baris data yang bernilai Nan atau kosong 
df.isnull().sum()

In [ ]:
# Mengubah nilai nan/kosong menjadi nilai mean
mean = df.mean()
df = df.replace(np.nan, mean)
df.head(10)

In [ ]:
# Mengubah/Encode kategori dari tipe data string menjadi tipe integer
from sklearn.preprocessing import OneHotEncoder 

df["gender"] = pd.factorize(df.gender) [0]
df["gender"] = pd.get_dummies(df["gender"])
df['ever_married'] = pd.get_dummies(df["ever_married"])
df["work_type"] = pd.get_dummies(df["work_type"])
df["Residence_type"] = pd.get_dummies(df["Residence_type"])
df["smoking_status"] = pd.get_dummies(df["smoking_status"])
df.head()

In [ ]:
# Menampilkan kembali desktriptif statistik yang telah di preprocessing
df.describe()

In [ ]:
# Setelah melakukan deskriptif statistik, kemudian dilakukan visualisasi data deskriptif untuk kolom yang telah di preprocessing
plt.figure(figsize = (30, 60))

# Melakukan visualisasi data deskritif untuk kolom gender
plt.subplot(8,5,1)
plt.axvline(df["gender"].mean(), color = "gray")
plt.axvline(df["gender"].median(), color = "purple", label = "median")
plt.title("Visualisasi Data Deskriptif Untuk Setiap gender", fontsize=10)
sns.distplot(df["gender"])

# Melakukan visualisasi data deskriptif untuk kolom age
plt.subplot(8,5,2)
plt.axvline(df["age"].mean(), label="mean", color="gray")
plt.axvline(df["age"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom Umur", fontsize=10)
sns.distplot(df["age"])

# Melakukan visualisasi data deskriptif untuk kolom hypertension
plt.subplot(8,5,3)
plt.axvline(df["hypertension"].mean(), label="mean", color="gray")
plt.axvline(df["hypertension"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom hypertension", fontsize=10)
sns.distplot(df["hypertension"])

# Melakukan visualisasi data deskriptif untuk kolom heart_disease
plt.subplot(8,5,4)
plt.axvline(df["heart_disease"].mean(), label="mean", color="gray")
plt.axvline(df["heart_disease"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom heart_disease", fontsize=10)
sns.distplot(df["heart_disease"])

# Melakukan visualisasi data deskriptif untuk kolom ever_married
plt.subplot(8,5,5)
plt.axvline(df["ever_married"].mean(), label="mean", color="gray")
plt.axvline(df["ever_married"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom ever_married", fontsize=10)
sns.distplot(df["ever_married"])

# Melakukan visualisasi data deskriptif untuk kolom work_type
plt.subplot(8,5,6)
plt.axvline(df["work_type"].mean(), label="mean", color="gray")
plt.axvline(df["work_type"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom work_type", fontsize=10)
sns.distplot(df["work_type"])

# Melakukan visualisasi data deskriptif untuk kolom Residence_type
plt.subplot(8,5,7)
plt.axvline(df["Residence_type"].mean(), label="mean", color="gray")
plt.axvline(df["Residence_type"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom Residence_type", fontsize=10)
sns.distplot(df["Residence_type"])

# Melakukan visualisasi data deskriptif untuk kolom avg_glucose_level
plt.subplot(8,5,8)
plt.axvline(df["avg_glucose_level"].mean(), label="mean", color="gray")
plt.axvline(df["avg_glucose_level"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom avg_glucose_level", fontsize=10)
sns.distplot(df["avg_glucose_level"])

# Melakukan visualisasi data deskriptif untuk kolom bmi
plt.subplot(8,5,9)
plt.axvline(df["bmi"].mean(), label="mean", color="gray")
plt.axvline(df["bmi"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom bmi", fontsize=10)
sns.distplot(df["bmi"])

# Melakukan visualisasi data deskriptif untuk kolom smoking status
plt.subplot(8,5,10)
plt.axvline(df["smoking_status"].mean(), label="mean", color="gray")
plt.axvline(df["smoking_status"].median(), label="median", color="purple")
plt.title("Visualisasi Data Deskriptif Untuk Kolom smoking_status", fontsize=10)
sns.distplot(df["smoking_status"])

In [ ]:
# Membuat heatmap korelasi dataset
plt.figure(figsize=(15, 10))

corr_data_stroke = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
corr_data_stroke.set_title("Korelasi Data Stroke Pre-Processing", fontdict={"fontsize":12}, pad=10)

In [ ]:
# Membagi data kedalam variabel x dan y
x = df.drop(["stroke"], axis=1)
y = df["stroke"]

In [ ]:
#Melakukan split data menjadi data train dan data test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)
print("Data Train : ", x_train.shape, y_train.shape)
print("Data Test :", x_test.shape, y_test.shape)

In [ ]:
#Melakukan penskalaan dan standarisasi fitur menggunakan StandardScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#Create a Gaussian Classifier
model = GaussianNB()
print(model)

In [ ]:
# Train the model using the training sets
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(acc)
print(cm)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier( criterion="gini", max_depth=4,random_state=1)

clf_tree.fit(x_train, y_train)

y_pred = clf_tree.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(acc)
print(cm)


In [ ]:
from sklearn.svm import SVC

from sklearn import svm
#create a classifier
cls = svm.SVC(kernel="linear")
#train the model
cls.fit(x_train, y_train)
#predict the response
pred = cls.predict(x_test)

from sklearn import metrics
#classification report
print(metrics.classification_report(y_test,y_pred=pred))